In [27]:
pwd

'/Users/ozkansafak/code/fraktur/notebooks'

In [26]:
%%time
%load_ext autoreload
%autoreload 2

# import standard Python modules
import time 
import numpy as np
import os, sys
import json
import re
import glob
import openai

# # Get the root path of the project
sys.path.append(os.path.abspath(".."))

# Import necessary modules from your src directory
from src.utils import timeit, encode_image, plt
from src.processing import compute_log_spectrum_1d, extract_image_bbox, save_images
from src.api_requests import construct_payload, send_gpt_request
from src.document_generation import save_document
from src.api_requests import single_page

from IPython.core.display import display, HTML
from IPython.display import clear_output
display(HTML("<style>.container { width:90% !important; }</style>"))

print('sys.executable:', sys.executable)
print('sys.version:', sys.version, '\n')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<timed exec>:23: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display


sys.executable: /Users/ozkansafak/code/fraktur/.venv/bin/python3
sys.version: 3.10.9 (main, Mar  1 2023, 12:20:14) [Clang 14.0.6 ] 

CPU times: user 1.69 ms, sys: 1.37 ms, total: 3.06 ms
Wall time: 2.29 ms


---
## Fraktur Translator

In [3]:
# Set the OpenAI API key from environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")

# Set the headers for the API request
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai.api_key}"
}

# Select the value of model_name in ["gpt-4o-2024-08-06", "gpt-4o-mini-2024-07-18"]
model_name = "gpt-4o-2024-08-06"
plotter = False

# Folder containing the Fraktur PDF files
image_path = "../data/Der Weltkrieg v8 Chap 4F (in Fraktur)/*pdf"

# Get the list of PDF file paths
fnames = sorted(glob.glob(image_path))
german_texts = {}
english_texts = {}
total_time = 0 

In [49]:
for i, fname in enumerate(fnames[1:2]):
    start = time.time()

    pageno = re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1)
    print(f"\n{'>'*10} page:{pageno} (i:{i}/{len(fnames)-1}) ", end="")
    
    raw_text, raw_german_text, german_text, english_text = single_page(fname, model_name, headers, plotter, pageno)
    
    german_texts[pageno] = german_text
    english_texts[pageno] = english_text
    delta = time.time() - start 
    total_time += delta
    print(f"[time:{delta:.2f}sec] [TotalRuntime:{int(total_time//60)} mins {(total_time - int(total_time//60)*60):.2f} sec] {'>'*10}\n")
    print(raw_text)



>>>>>>>>>> page:457 (i:0/83) [time:22.92sec] [TotalRuntime:40 mins 33.38 sec] >>>>>>>>>>

<raw_german>
Aufgabe und Gliederung der Njemen-Armee.
Die N j e m e n - A r m e e unter General der Infanterie O t t o
v o n B e l o w deckte zu dieser Zeit mit rund sieben Infanterie-Divisionen
und fünf Kavallerie-Divisionen1) den Raum nördlich des Njemen von der
unteren Dubissa bis in die Gegend östlich von Libau in einer Frontbreite
von etwa 250 Kilometern. Der gegenüberstehende Feind schien an Zahl
etwas überlegen2). General von Below wollte den Angriff, ähnlich wie es
der Oberbefehlshaber Ost seinerzeit in der Weisung vom 14. Juni3) ange-
ordnet hatte, unter Vermeidung der starken feindlichen Stellungen bei Schau-
len gegen den vorwiegend aus Kavallerie bestehenden russischen Nordflügel
führen, um dann gegen Flanke und Rücken der Schaulen-Stellung einzu-
schwenken. Dementsprechend gliederte er seine Truppen unter Schwächung
des rechten Flügels wie folgt:
S ü d g r u p p e unter Generalleutna

In [33]:
# # save json outputs
# folder_name = 'third_attempt'
# with open(f'../output/{folder_name}/english_texts.json', 'w') as f:
#     json.dump(english_texts, f)
# with open(f'../output/{folder_name}/german_texts.json', 'w') as f:
#     json.dump(german_texts, f)

doc1, fname1 = save_document(german_texts, folder_name, language='German')
doc2, fname2 = save_document(english_texts, folder_name, language='English')


``` 
1.  Upload Input folder of pdfs to blob storage.
2.  Read file from s3.
3.  FFT in y -> (x_hi, x_lo), write half_cropped_image to s3
4.  FFT in x -> (y_hi, y_lo), write cropped_image to s3
5.  Read cropped image from s3 -> encode_image -> translate and transcribe -> JSON output

```

### Available models and pricing:
```
"gpt-4o-2024-08-06":
    "price_txt": "$2.50 / 1M input tokens"
    "price_img": "$0.001913 / 1500px^2"
    
"gpt-4o-mini-2024-07-18":
    "price_txt": "$0.150 / 1M input tokens"
    "price_img": "$0.003825 / 1500px^2"
    
```

In [35]:
import pypandoc

# Save your .docx file
fname = 'output_filename'
doc1.save(f'{fname}.docx')

# Convert the .docx file to .pdf
output_pdf = pypandoc.convert_file(f'{fname}.docx', 'pdf', outputfile=f'{fname}.pdf')
assert output_pdf == ""


RuntimeError: Pandoc died with exitcode "47" during conversion: pdflatex not found. Please select a different --pdf-engine or install pdflatex


In [47]:
output_pdf = pypandoc.convert_file(
    f'{fname}.docx', 
    'pdf', 
    outputfile=f'{fname}.pdf', 
    extra_args=['--pdf-engine=wkhtmltopdf']
)


RuntimeError: Pandoc died with exitcode "4" during conversion: pdf-engine wkhtmltopdf is not compatible with output format latex


In [44]:
os.listdir()

['.DS_Store',
 'output_filename.docx',
 '.ipynb_checkpoints',
 'main_fraktur.ipynb']